In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv
import random
from tqdm.notebook import tqdm as tqdm 
import numpy as np
import pandas as pd 
import json
import datetime
from scipy import stats
import time
from datetime import timedelta
import re
import string

In [243]:
courses = pd.read_csv('courses.csv')

In [245]:
courses.head()

,certificate_availability,cost,course_link,course_name,duration,language,overview,reviews,site,stars
0,Certificate Available,Free Online Course,https://www.classcentral.com/course/independen...,Elements of AI,"6 weeks long, 5-10 hours a week",English,Overview The goal of this course is to demyst...,641 reviews,classcentral,Five out of five stars
1,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/machine-le...,Machine Learning,"11 weeks long, 60 hours worth of material",English,Overview Class Central Tips Learn ...,375 reviews,classcentral,Five out of five stars
2,Paid Certificate Available,Free Online Course,https://www.classcentral.com/course/swayam-cyb...,Cyber Security,15 weeks long,English,Overview 1. Why this Course on Cybe...,236 reviews,classcentral,Four and a half out of five stars
3,$149.00 Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/edx-unlock...,Unlocking Information Security I: From Cryptog...,"5 weeks long, 4-6 hours a week",English,Overview Information Security is everywhere:...,236 reviews,classcentral,Five out of five stars
4,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/informatio...,"Information Systems Auditing, Controls and Ass...","4 weeks long, 8 hours worth of material",English,Overview Class Central Tips Learn ...,208 reviews,classcentral,Four and a half out of five stars


In [248]:
courses.shape

(4348, 10)

In [86]:
missing = np.nan
df = pd.DataFrame()
for i in tqdm(range(87)):
    response = requests.get('https://www.classcentral.com/subject/cs?page='+ f'{i+1}', timeout= 15)
    sleep(random.uniform(1,2))
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tags = soup.find_all('td', class_ = 'width-12-16 large-up-width-8-16 xxlarge-up-width-9-16 relative')
        final_tags = []
        for tag in tags:
            if not tag.get('data-track-impression'):
                final_tags.append(tag)
        for final_tag in final_tags:
            site = 'classcentral'
            try:
               course_name = final_tag.find('span', class_='text-1 weight-semi line-tight').text.strip()
            except:
                course_name = missing
            try:
                course_link = ('https://www.classcentral.com'+final_tag.find('a', class_='color-charcoal block line-tight course-name').get('href'))
            except:
                course_link = missing
            temp_dic1 = {'site' : site,
                  'course_name' : course_name,
                  'course_link' : course_link,}
            df =(df.append(temp_dic1, ignore_index=True))
    else:
        print('Response Status in page links not 200')

In [237]:
courses = pd.DataFrame()

In [238]:
for index, row in tqdm(df.iterrows()):
    course_link = row['course_link']
    try:
        response = requests.get(course_link, timeout= 15)
        sleep(random.uniform(2,3))
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            info = soup.find('div', class_= 'bg-white small-down-padding-medium padding-large border-all border-gray-light radius-small margin-bottom-medium')
            lis = info.find_all('li', class_ = 'nowrap padding-horz-small padding-vert-xsmall border-bottom border-gray-light row vert-align-top horz-align-left')
            header = soup.find('header', class_='bg-white small-down-padding-medium padding-large radius-small border-all border-gray-light cmpt-grid-header margin-bottom-medium')
            overviewblock = soup.find('div', class_='bg-white border-all border-gray-light radius-small small-down-padding-medium padding-large margin-bottom-medium relative')
            try:
                for li in lis:
                    if 'language' in li.find('i').get('class')[0]:
                        language = li.find('a').text.strip()
            except:
                language = missing
            try:
                for li in lis:
                    if 'dollar' in li.find('i').get('class')[0]:
                        cost = li.find('span').text.strip()
            except:
                cost = missing
            try:
                for li in lis:
                    if 'credential' in li.find('i').get('class')[0]:
                        certificate_availability = li.find('span').text.strip()
            except:
                certificate_availability = missing
            try:
                for li in lis:
                    if 'clock' in li.find('i').get('class')[0]:
                        duration = li.find('span').text.strip()
            except:
                duration = missing
            try:
                stars = header.find('span', class_= 'cmpt-rating-xlarge').get('aria-label')
            except:
                stars = missing
            try:
                reviews = header.find('span', class_= 'margin-left-xxsmall').text.strip()
            except:
                reviews = missing
            try:
                overview = overviewblock.text.strip().replace('\n', ' ').replace('\xa02 ', ' ').replace('/', ' ').replace('\xa0', ' ')
            except:
                overview = missing
            temp_dic2  = {
                          'site' : row['site'],
                          'course_name' : row['course_name'],
                          'course_link' : row['course_link'],
                          'language' : language,
                          'cost' : cost,
                          'certificate_availability' : certificate_availability,
                          'duration' : duration,
                          'stars' : stars,
                          'reviews' : reviews,
                          'overview' : overview}
            courses = courses.append(temp_dic2, ignore_index=True)
        else:
            print('Response Status in page links not 200')
    except:
        print('Connection error')

In [240]:
courses.to_csv('courses.csv', index=False)

In [239]:
courses

,certificate_availability,cost,course_link,course_name,duration,language,overview,reviews,site,stars
0,Certificate Available,Free Online Course,https://www.classcentral.com/course/independen...,Elements of AI,"6 weeks long, 5-10 hours a week",English,Overview The goal of this course is to demyst...,641 reviews,classcentral,Five out of five stars
1,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/machine-le...,Machine Learning,"11 weeks long, 60 hours worth of material",English,Overview Class Central Tips Learn ...,375 reviews,classcentral,Five out of five stars
2,Paid Certificate Available,Free Online Course,https://www.classcentral.com/course/swayam-cyb...,Cyber Security,15 weeks long,English,Overview 1. Why this Course on Cybe...,236 reviews,classcentral,Four and a half out of five stars
3,$149.00 Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/edx-unlock...,Unlocking Information Security I: From Cryptog...,"5 weeks long, 4-6 hours a week",English,Overview Information Security is everywhere:...,236 reviews,classcentral,Five out of five stars
4,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/informatio...,"Information Systems Auditing, Controls and Ass...","4 weeks long, 8 hours worth of material",English,Overview Class Central Tips Learn ...,208 reviews,classcentral,Four and a half out of five stars
...,...,...,...,...,...,...,...,...,...,...
4343,Certificate Available,Free Online Course,https://www.classcentral.com/course/openhpi-in...,Internetworking with TCP/IP,6 weeks long,English,Overview The Internet connects more than half...,0 reviews,classcentral,Zero out of five stars
4344,Certificate Available,Free Online Course,https://www.classcentral.com/course/france-uni...,Deep Learning,"6 weeks long, 3 hours a week",French,Overview À propos du cours Vous êtes passionn...,0 reviews,classcentral,Zero out of five stars
4345,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/gcp-big-da...,Google Cloud Platform Big Data and Machine Lea...,"6 weeks long, 6-10 hours a week",German,Overview Class Central Tips Learn ...,0 reviews,classcentral,Zero out of five stars
4346,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/gcp-fundam...,Google Cloud Platform Fundamentals: Core Infra...,"1 week long, 6-10 hours a week",German,Overview Class Central Tips Learn ...,0 reviews,classcentral,Zero out of five stars


In [242]:
courses['overview'][4000]

'Overview            Building on the core ideas presented in Distributed Machine Learning with Spark, this course covers advanced topics for training and deploying large-scale learning pipelines. You will study state-of-the-art distributed algorithms for collaborative filtering, ensemble methods (e.g., random forests), clustering and topic modeling, with a focus on model parallelism and the crucial tradeoffs between computation and communication.  After completing this course, you will have a thorough understanding of the statistical and algorithmic principles required to develop and deploy distributed machine learning pipelines. You will further have the expertise to write efficient and scalable code in Spark, using MLlib and the spark.ml package in particular.             Taught by Ameet Talwalkar and Jon Bates   Tags                      usa'

In [194]:
df['course_link'][1000]

'https://www.classcentral.com/course/opensap-sap-s-4hana-in-a-nutshell-3252'